In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.model_selection import GridSearchCV,train_test_split

In [2]:
def metricas(model,Xt,Xv,yt,yv):
    print "ROC train:%.3f | ROC test:%.3f " %(roc_auc_score(y_true=yt,y_score=model.predict_proba(Xt)[:,1]),roc_auc_score(y_true=yv,y_score=model.predict_proba(Xv)[:,1]))
    print "ACC train:%.3f | ACC test:%.3f " %(accuracy_score(y_true=yt,y_pred=model.predict(Xt)),accuracy_score(y_true=yv,y_pred=model.predict(Xv)))

In [3]:
df = pickle.load(open('churn_con_woe','rb'))

In [4]:
df.head()

,V_ACCT_LGHT,C_IP,C_VMP,V_N_VM_MESS,V_T_DAY_CHG,V_T_EVE_CALLS,V_T_EVE_CHG,V_T_NIG_CALLS,V_T_NIG_CHG,V_T_INT_CALLS,...,W_V_ACCT_LGHT,W_V_N_VM_MESS,W_V_T_DAY_CHG,W_V_T_EVE_CALLS,W_V_T_EVE_CHG,W_V_T_NIG_CALLS,W_V_T_NIG_CHG,W_V_T_INT_CALLS,W_V_T_INT_CHG,W_V_CS_CALLS
0,128,No,Yes,25,45.07,99,16.78,91,11.01,3,...,0.061033,1.219037,-1.864202,0.068283,0.065696,-0.089148,-0.05179,-0.29035,0.007263,0.295956
1,156,No,No,0,29.67,116,16.78,86,10.73,2,...,0.061033,-0.147174,0.561314,-0.053201,0.065696,-0.089148,-0.05179,-0.29035,0.007263,0.295956
2,57,No,No,0,14.60,127,16.48,93,10.42,2,...,0.053118,-0.147174,0.178109,-0.053201,0.065696,-0.089148,-0.05179,-0.29035,0.007263,0.295956
3,58,No,Yes,20,33.07,89,18.16,92,10.65,2,...,0.053118,1.219037,0.561314,0.031576,0.065696,-0.089148,-0.05179,-0.29035,0.007263,0.295956
4,149,No,No,0,31.89,81,17.12,79,11.89,1,...,0.061033,-0.147174,0.561314,0.031576,0.065696,-0.089148,-0.05179,-0.29035,0.007263,0.295956


In [5]:
X = df[[v for v in df.columns if v[:2]=='V_']+[v for v in df.columns if v[:4]=='W_C_']].copy()
y = df['TARGET'].copy()

In [6]:
Xt,Xv,yt,yv = train_test_split(X,y,train_size=0.7)

In [7]:
model = DecisionTreeClassifier()

In [8]:
model.fit(Xt,yt)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [9]:
metricas(model,Xt,Xv,yt,yv)

ROC train:1.000 | ROC test:0.865 
ACC train:1.000 | ACC test:0.930 


In [10]:
param_grid = dict(criterion = ['gini','entropy'],
splitter = ['best','random'],
max_features = range(2,13),
max_depth = range(3,15))

In [11]:
model = DecisionTreeClassifier()

In [14]:
grid = GridSearchCV(cv=3,
                    verbose=True,
                    estimator=model,
                    param_grid=param_grid,
                    n_jobs=-1,
                    scoring='roc_auc')

In [17]:
%%time
grid.fit(X,y)

Fitting 3 folds for each of 528 candidates, totalling 1584 fits


[Parallel(n_jobs=-1)]: Done 780 tasks      | elapsed:    3.2s


CPU times: user 3.23 s, sys: 236 ms, total: 3.47 s
Wall time: 6.68 s


[Parallel(n_jobs=-1)]: Done 1584 out of 1584 | elapsed:    6.5s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_features': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 'splitter': ['best', 'random'], 'criterion': ['gini', 'entropy'], 'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=True)

In [19]:
model = grid.best_estimator_
model.fit(Xt,yt)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=11, max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [20]:
metricas(model,Xt,Xv,yt,yv)

ROC train:0.912 | ROC test:0.902 
ACC train:0.952 | ACC test:0.935 
